In [ ]:
%load_ext autoreload
%autoreload 2

import env

import pandas as pd
import json
import subprocess
import sys
from datetime import datetime
from pathlib import Path

# --- 1. Загрузка конфигурации ---
service = env.get_gservice()

if service:
    df_sheet = env.read_df_from_spreadsheet(service, env.SHEET_ID, env.SHEET_NAME)
    print("Данные из Google Sheets загружены")
else:
    raise ConnectionError("Не удалось подключиться к Google API")

# --- 2. Маппинг названий проверок на файлы ноутбуков ---
NOTEBOOK_MAP = {
    '01-incent.cr': '01-incent.opex.check_conversion_rates.ipynb',
    '02-incent.p7': '02-incent.opex.check_payers7.ipynb',
    # Добавляйте новые проверки здесь:
    # '03-incent.ltv': '03-incent.opex.check_ltv.ipynb',
}

# Директория с ноутбуками
NOTEBOOKS_DIR = Path.cwd()

# --- 3. Определение текущего дня недели ---
# 0 = Monday, 1 = Tuesday, ..., 6 = Sunday
DAY_NAMES = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']

today = datetime.now()
current_day_idx = today.weekday()
current_day = DAY_NAMES[current_day_idx]

print(f"Сегодня: {today.strftime('%Y-%m-%d')} ({current_day.upper()})")
print(f"="*50)


# --- 4. Функция парсинга поля run ---
def should_run_today(run_config, current_day):
    """
    Проверяет, нужно ли запускать проверку сегодня.
    
    Форматы поля run:
    - "daily" или "*" — каждый день
    - "mon,wed,fri" — список дней через запятую
    - ["mon", "wed", "fri"] — JSON массив дней
    - "weekdays" — пн-пт
    - "weekends" — сб-вс
    
    Returns: True если нужно запускать, False если нет
    """
    if pd.isna(run_config) or run_config == '':
        return False
    
    run_str = str(run_config).strip().lower()
    
    # Специальные значения
    if run_str in ['daily', '*', 'everyday']:
        return True
    
    if run_str == 'weekdays':
        return current_day in ['mon', 'tue', 'wed', 'thu', 'fri']
    
    if run_str == 'weekends':
        return current_day in ['sat', 'sun']
    
    # Попытка распарсить как JSON массив
    try:
        days_list = json.loads(run_str)
        if isinstance(days_list, list):
            days_list = [d.lower().strip() for d in days_list]
            return current_day in days_list
    except (json.JSONDecodeError, TypeError):
        pass
    
    # Парсинг как строка через запятую: "mon,wed,fri"
    days_list = [d.strip().lower() for d in run_str.split(',')]
    return current_day in days_list


# --- 5. Функция запуска ноутбука ---
def run_notebook(notebook_name, check_name):
    """
    Запускает ноутбук через papermill.
    
    Args:
        notebook_name: имя файла ноутбука
        check_name: название проверки (для логирования)
    
    Returns:
        True если успешно, False если ошибка
    """
    notebook_path = NOTEBOOKS_DIR / notebook_name
    
    if not notebook_path.exists():
        print(f"  [ERROR] Файл не найден: {notebook_path}")
        return False
    
    print(f"  Запуск: {notebook_name}...")
    
    try:
        # Запуск через papermill (без сохранения output)
        result = subprocess.run(
            [
                sys.executable, '-m', 'papermill',
                str(notebook_path),
                '/dev/null',  # Не сохраняем output notebook
                '--log-output',
                '--no-progress-bar'
            ],
            capture_output=True,
            text=True,
            timeout=600  # 10 минут таймаут
        )
        
        if result.returncode == 0:
            print(f"  [OK] {check_name} завершён успешно")
            return True
        else:
            print(f"  [FAILED] {check_name} завершился с ошибкой:")
            print(result.stderr[:500] if result.stderr else "No error output")
            return False
            
    except subprocess.TimeoutExpired:
        print(f"  [TIMEOUT] {check_name} превысил лимит времени (10 мин)")
        return False
    except Exception as e:
        print(f"  [ERROR] {check_name}: {e}")
        return False


# --- 6. Основной цикл запуска ---
results = []

for _, row in df_sheet.iterrows():
    check_name = row['name']
    active_flag = row.get('active_flag', 'Disabled')
    run_config = row.get('run', '')
    
    # Пропускаем если нет в маппинге
    if check_name not in NOTEBOOK_MAP:
        continue
    
    notebook_name = NOTEBOOK_MAP[check_name]
    
    # Проверяем активность
    if active_flag != 'Enabled':
        print(f"\n[SKIP] {check_name}: отключен (active_flag != Enabled)")
        results.append({'check': check_name, 'status': 'DISABLED', 'reason': 'active_flag'})
        continue
    
    # Проверяем расписание
    if not should_run_today(run_config, current_day):
        print(f"\n[SKIP] {check_name}: не запланирован на {current_day.upper()} (run={run_config})")
        results.append({'check': check_name, 'status': 'SKIPPED', 'reason': f'not scheduled for {current_day}'})
        continue
    
    # Запускаем проверку
    print(f"\n[RUN] {check_name} (run={run_config})")
    success = run_notebook(notebook_name, check_name)
    
    results.append({
        'check': check_name,
        'status': 'SUCCESS' if success else 'FAILED',
        'reason': None
    })


# --- 7. Итоговый отчёт ---
print(f"\n{'='*50}")
print("ИТОГИ ЗАПУСКА:")
print(f"{'='*50}")

df_results = pd.DataFrame(results)

if not df_results.empty:
    for _, r in df_results.iterrows():
        status_icon = {
            'SUCCESS': '✓',
            'FAILED': '✗',
            'SKIPPED': '○',
            'DISABLED': '−'
        }.get(r['status'], '?')
        
        reason_str = f" ({r['reason']})" if r['reason'] else ""
        print(f"  {status_icon} {r['check']}: {r['status']}{reason_str}")
    
    # Статистика
    print(f"\nВсего: {len(df_results)} проверок")
    print(f"  Успешно: {(df_results['status'] == 'SUCCESS').sum()}")
    print(f"  Ошибки: {(df_results['status'] == 'FAILED').sum()}")
    print(f"  Пропущено: {(df_results['status'] == 'SKIPPED').sum()}")
    print(f"  Отключено: {(df_results['status'] == 'DISABLED').sum()}")
else:
    print("  Нет проверок для запуска.")

print(f"\nЗавершено: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")